# Steel-wire data reconstruction demo

Reproduce all steel-wire data figures.

Tested with CIL version 21.0.0 on Linux.

In [ ]:
# Import all CIL components needed
from cil.framework import ImageData, ImageGeometry
from cil.framework import AcquisitionGeometry, AcquisitionData
from cil.framework import BlockDataContainer

from cil.optimisation.algorithms import CGLS, SIRT, GD, FISTA, PDHG
from cil.optimisation.operators import BlockOperator, GradientOperator, IdentityOperator, \
                                       GradientOperator, FiniteDifferenceOperator
from cil.optimisation.functions import IndicatorBox, MixedL21Norm, L2NormSquared, \
                                       BlockFunction, L1Norm, LeastSquares, \
                                       OperatorCompositionFunction, TotalVariation, \
                                       ZeroFunction

# CIL Processors
from cil.processors import CentreOfRotationCorrector, Slicer

from cil.utilities.display import show2D

# Import from cil.plugins.astra
from cil.plugins.astra.processors import FBP, AstraBackProjector3D
from cil.plugins.astra.operators import ProjectionOperator, AstraProjector3DSimple

# All external imports
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import scipy

In [ ]:
from cil.utilities.dataexample import SYNCHROTRON_PARALLEL_BEAM_DATA
data = SYNCHROTRON_PARALLEL_BEAM_DATA.get()

In [ ]:
plt.figure()
show2D(data.get_slice(angle=0),cmap='inferno',origin='upper-left')

In [ ]:
data_raw20 = data.get_slice(vertical=20)
scale = data_raw20.sum()/data_raw20.size
print(scale)

In [ ]:
data = data / scale

In [ ]:
data.log(out=data)
data *= -1

In [ ]:
data = CentreOfRotationCorrector.xcorr(slice_index='centre')(data)

In [ ]:
ag_create = AcquisitionGeometry.create_Parallel3D() \
    .set_panel(num_pixels=(160, 135)) \
    .set_angles(angles=np.linspace(-88.2, 91.8, 91))

In [ ]:
sx = 44
sz = 103

In [ ]:
data90 = Slicer(roi={'angle':(0,90), 'horizontal': (20,140,1)})(data)

In [ ]:
print(data90)

In [ ]:
plt.figure()
show2D(data90.subset(angle=0),cmap='inferno',origin='upper-left')

plt.figure()
show2D(data90.subset(vertical=sz),cmap='inferno',origin='upper-left')

In [ ]:
data15 = Slicer(roi={'angle': (0,90,6)})(data90)

In [ ]:
plt.figure()
show2D(data15.subset(vertical=sz),cmap='inferno',origin='upper-left')

In [ ]:
data90.reorder(order='astra')
data15.reorder(order='astra')

In [ ]:
ag = data15.geometry
ig = ag.get_ImageGeometry()

In [ ]:
recon = FBP(ig, ag, device='gpu')(data15)

In [ ]:
ca1 = -0.01
ca2 =  0.11

In [ ]:
plt.figure()
show2D(recon.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(recon.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
ag90 = data90.geometry
recon90 = FBP(ig, ag90, device='gpu')(data90)

In [ ]:
plt.figure()
show2D(recon90.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(recon90.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
A = ProjectionOperator(ig, ag)

In [ ]:
x0 = ig.allocate(0.0)
b = data15

In [ ]:
myCGLS = CGLS(x_init=x0, operator=A, data=b, max_iteration=1000)

In [ ]:
myCGLS.run(2,verbose=True)

In [ ]:
plt.figure()
show2D(myCGLS.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(myCGLS.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
myCGLS.run(18,verbose=1)

In [ ]:
plt.figure()
show2D(myCGLS.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(myCGLS.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
myCGLS.run(180,verbose=True)

In [ ]:
plt.figure()
show2D(myCGLS.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(myCGLS.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
mySIRT = SIRT(x_init=x0, operator=A, data=b, lower=0.0, upper=0.09, tolerance=0, max_iteration=1000)
mySIRT.run(200,verbose=1)

In [ ]:
plt.figure()
show2D(mySIRT.solution.subset(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(mySIRT.solution.subset(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
D = IdentityOperator(ig)
alpha = 1.0

Atilde = BlockOperator(A, alpha * D)
z = D.range.allocate(0)
btilde = BlockDataContainer(b, z)

myTikhonovI = CGLS(x_init=x0, operator=Atilde, data=btilde, max_iteration=1000)
myTikhonovI.run(1000,verbose=1)

In [ ]:
plt.figure()
show2D(myTikhonovI.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(myTikhonovI.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
D = GradientOperator(ig)
alpha = 1.0

Atilde = BlockOperator(A, alpha*D, shape=(2,1))
btilde = BlockDataContainer(b, D.range.allocate(0))

myTikhonovD = CGLS(x_init=x0, operator=Atilde, data=btilde, \
                   max_iteration=1000, update_objective_interval = 10)
myTikhonovD.run(1000, verbose=1)

In [ ]:
plt.figure()
show2D(myTikhonovD.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(myTikhonovD.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
az = 0.1
ay = 30
ax = 30

fdz = FiniteDifferenceOperator(ig, direction='vertical', bnd_cond='Neumann')
fdy = FiniteDifferenceOperator(ig, direction='horizontal_y', bnd_cond='Neumann')
fdx = FiniteDifferenceOperator(ig, direction='horizontal_x', bnd_cond='Neumann')

Atilde = BlockOperator(A, ax*fdx, ay*fdy, az*fdz, shape=(4,1))
btilde = BlockDataContainer(b, fdx.range.allocate(0.0), fdy.range.allocate(0.0), fdz.range.allocate(0.0))

myTikhonovHorz = CGLS(x_init=x0, operator=Atilde, data=btilde, update_objective_interval = 10, \
                      max_iteration = 1000)

In [ ]:
myTikhonovHorz.run(100, verbose=1)

In [ ]:
plt.figure()
show2D(myTikhonovHorz.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(myTikhonovHorz.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
az = 60
ay = 0.1
ax = 0.1

Atilde = BlockOperator(A, ax*fdx, ay*fdy, az*fdz, shape=(4,1))
btilde = BlockDataContainer(b, fdx.range.allocate(0.0), fdy.range.allocate(0.0), fdz.range.allocate(0.0))

myTikhonovVert = CGLS(x_init=x0, operator=Atilde, data=btilde, update_objective_interval=10, \
                     max_iteration=1000)

In [ ]:
myTikhonovVert.run(100, verbose=1)

In [ ]:
plt.figure()
show2D(myTikhonovVert.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(myTikhonovVert.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
f1 = LeastSquares(A, b)
f2 = OperatorCompositionFunction(L2NormSquared(),D)
f = f1 + (alpha**2)*f2

myGD = GD(x_init=x0, objective_function=f, step_size=None, max_iteration=1000, update_objective_interval = 10)
myGD.run(1000, verbose=1)

In [ ]:
plt.figure()
show2D(myGD.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(myGD.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
F = LeastSquares(A, b)
G = IndicatorBox(lower=0.0)
myFISTANN = FISTA(f=F, g=G, x_init=x0, max_iteration=1000)
myFISTANN.run(300, verbose=1)

In [ ]:
plt.figure()
show2D(myFISTANN.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(myFISTANN.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
a1 = 30
G = a1*L1Norm()

myFISTAL1 = FISTA(f=F, g=G, x_init=x0, max_iteration=1000, update_objective_interval=10)

In [ ]:
myFISTAL1.run(300,verbose=1)

In [ ]:
plt.figure()
show2D(myFISTAL1.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(myFISTAL1.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
#GTV = TotalVariation(0.02,100,1e-6)
GTV = 0.02*TotalVariation()

In [ ]:
myFISTATV = FISTA(f=F, g=GTV, x_init=x0 ,max_iteration=10000)

In [ ]:
myFISTATV.run(200,verbose=1)

In [ ]:
plt.figure()
show2D(myFISTATV.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(myFISTATV.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
alpha = 0.02
F = BlockFunction(L2NormSquared(b=b), alpha*MixedL21Norm())
K = BlockOperator(A, GradientOperator(ig))
G = ZeroFunction()
myPDHG = PDHG(f=F, g=G, operator=K, max_iteration=10000)

In [ ]:
myPDHG.run(5000,verbose=2)

In [ ]:
plt.figure()
show2D(myPDHG.solution.get_slice(horizontal_x=sx),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')
plt.figure()
show2D(myPDHG.solution.get_slice(vertical=sz),cmap='inferno',fix_range=(ca1,ca2),origin='upper-left')

In [ ]:
plt.figure()
plt.loglog(np.arange(1,len(myFISTATV.objective)),  myFISTATV.objective[1:])
plt.loglog(np.arange(1,len(myPDHG.objective)),myPDHG.objective[1:])
plt.loglog(np.arange(1,len(myPDHG.objective)),myPDHG.dual_objective[1:])
plt.loglog(np.arange(1,len(myPDHG.objective)),np.array(myPDHG.objective[1:])-np.array(myPDHG.dual_objective[1:]))
plt.ylim((1e0,1e5))
plt.legend(['FISTA','PDHG primal','PDHG dual','PDHG gap'])
plt.grid()
plt.xlabel('Number of iterations')
plt.ylabel('Objective value')